In [415]:
import json
from functools import cmp_to_key
from inspect import getsource
from sys import argv
import re

complete_match = ["""import json as a,sys
b=sys.argv
a.dump(dict(zip(*[sorted(x)for x in a.load(open(b[1]))])),open(b[2],'w'))""",
                      """import json as a,sys,re
b=sys.argv
a.dump(dict(zip(*[sorted(i,key=lambda y:[int(c)if c.isdigit()else c for c in re.split('(\d+)',y)])for i in a.load(open(b[1]))])),open(b[2],'w'))""",
                      """import json as a,sys,re
def dimensions(string,i):
    it=list(re.finditer('\[([0-9]+)\]'*i,string))
    return (0,0) if not i else (i,it) if len(it) else dimensions(string,i-1)
def order_key(ori):
    K=ori[:]
    order=re.findall('\$[a-zA-Z](\d+)',ori)
    if order:
        d,ds = dimensions(ori,2)
        o=int(order[0])
        if d:
            K = ori[:ds[0].start()]+ds[-1 if len(ds) < o else o-1].group(0)+order[0]
        else:
            x_dim = list(re.finditer('[a-zA-Z]\d+\_\d+',ori))
            K= ori[:x_dim[0].start()]+x_dim[-1 if len(x_dim) < o else o-1].group(0)+order[0] if x_dim else ori
    else:
        K = ori
    return [int(c)if c.isdigit()else c for c in re.split('(\d+)',K)]
A=[ sorted(i) for i in a.load(open(b[1]))]
X=[[order_key(j)for j in i] for i in A]
Y=[[*zip(A[i],X[i])] for i in[0,1]]
Y=[sorted(x,key=lambda y:y[1]) for x in Y]
A=[[z[0]for z in y]for y in Y]
a.dump(dict(zip(*A)),open(b[2],'w'))"""]
    
incomplete_match = ["""import json as a,sys
b=sys.argv
A=[sorted(x)for x in a.load(open(b[1]))]
t={}
B=A[1][:]
D=[]
C=0
for i in range(0,len(t),2):
    if type(t[i])!=list:
        D+=[B[t[i]+C+j]for j in range(t[i+1])]
        C+=t[i+1]
    else:
        D+=[B[t[i][j]+C+j]for j in range(len(t[i]))]
        C+=len(t[i])
A[1]=D
a.dump(dict(zip(*A)),open(b[2],'w'))""",
                        '''import json as a,sys,re
t={}
b=sys.argv
A=[sorted(i,key=lambda y:[int(c)if c.isdigit()else c for c in re.split('(\d+)',y)])for i in a.load(open(b[1]))]
B=A[1][:]
D=[]
C=0
for i in range(0,len(t),2):
    if type(t[i])!=list:
        D+=[B[t[i]+C+j]for j in range(t[i+1])]
        C+=t[i+1]
    else:
        D+=[B[t[i][j]+C+j]for j in range(len(t[i]))]
        C+=len(t[i])
A[1]=D
a.dump(dict(zip(*A)),open(b[2],'w'))''',
                         """import json as a,sys,re
def dimensions(string,i):
    it=list(re.finditer('\[([0-9]+)\]'*i,string))
    return (0,0) if not i else (i,it) if len(it) else dimensions(string,i-1)
def order_key(ori):
    K=ori[:]
    order=re.findall('\$[a-zA-Z](\d+)',ori)
    if order:
        d,ds = dimensions(ori,2)
        o=int(order[0])
        if d:
            K = ori[:ds[0].start()]+ds[-1 if len(ds) < o else o-1].group(0)+order[0]
        else:
            x_dim = list(re.finditer('[a-zA-Z]\d+\_\d+',ori))
            K= ori[:x_dim[0].start()]+x_dim[-1 if len(x_dim) < o else o-1].group(0)+order[0] if x_dim else ori
    else:
        K = ori
    return [int(c)if c.isdigit()else c for c in re.split('(\d+)',K)]
t={}
b=sys.argv
A=[ sorted(i) for i in a.load(open(b[1]))]
X=[[order_key(j)for j in i] for i in A]
Y=[list(zip(A[i],X[i])) for i in[0,1]]
Y=[sorted(x,key=lambda y:y[1]) for x in Y]
A=[[z[0]for z in y]for y in Y]
B=A[1][:]
D=[]
C=0
for i in range(0,len(t),2):
    if type(t[i])!=list:
        D+=[B[t[i]+C+j]for j in range(t[i+1])]
        C+=t[i+1]
    else:
        D+=[B[t[i][j]+C+j]for j in range(len(t[i]))]
        C+=len(t[i])
A[1]=D
a.dump(dict(zip(*A)),open(b[2],'w'))"""]

def shift_count(ori,new):
    count = 0
    for i in range(len(ori)):
        if ori[new[0][i]] != new[1][i]:
            count += 1
    return count

def generate_shift_table(ori,new):
    idx=[]
    for i in range(len(ori)):
        if ori[new[0][i]] != new[1][i]:
            idx.append(new[1].index(ori[new[0][i]])-i)
        else:
            idx.append(0)
    return idx

def generate_run_length_table(idx):
    # [initialization]
    # add end flag
    idx += ['*']  
    # current run
    current = idx[0] 
    # current length
    run_length = 0
    # final output info
    res = []
    # readble output info
    readable = []
    
    # [run_length]
    for i in range(len(idx)):
        if idx[i] == current:
            run_length += 1
        else:
            res += [current,run_length]
            readable.append([current,run_length])
            current = idx[i]
            run_length = 1
    return res, readable

def two_level_run_length(table):
    flag = False
    temp = []
    tltable = []
    for i in range(0,len(table),2):
        if not flag and table[i+1] == 1:
            temp = [[table[i]],1]
            flag = True
        elif flag and table[i+1] == 1:
            temp[0] += [table[i]]
        else:
            flag = False
            tltable += temp
            tltable += table[i:i+2]
            temp = []
    tltable += temp
    return tltable

def generate_tables(origin_table, input_table, method):
    methods = ['Default', 'Natural', 'Order']
    choise, idx, res, readable, tltable = -1, [], [], [], []
    ns = shift_count(origin_table, input_table)
    if ns == 0:
        print('%s key can sovle this case.' % method)
        choise = methods.index(method)
    else:
        print('%s key can\'t sovle this case.' % method)
        print('Shift count in this method: %d.' % ns)
        idx = generate_shift_table(origin_table,input_table)
        res, readable = generate_run_length_table(idx)
        tltable = two_level_run_length(res)
        print('Shift table string length: %d.' % len(str(idx)))
        print('One level run-length table string length: %d.' % len("".join(str(res).split(' '))))
        print('The most large consecutive segment length: %d.' % max(list(zip(*readable))[1]))
        print('Two level run-length table string length: %d.' % len("".join(str(tltable).split(' '))))
    return choise, idx, res, readable, tltable

def default_key_method(origin_mapping):
    keys, values = zip(*origin_mapping.items())
    keys, values = sorted(keys), sorted(values)
    return [keys,values]

def natural_key(x):
    return [int(c)if c.isdigit()else c for c in re.split('(\d+)',x)]

def natural_key_method(origin_mapping):
    ori = [list(origin_mapping.keys()),list(origin_mapping.values())]
    return [sorted(x,key=natural_key) for x in ori]

def dimensions(string,i):
    it=list(re.finditer('\[([0-9]+)\]'*i,string))
    return (0,0) if not i else (i,it) if len(it) else dimensions(string,i-1)

def order_key(ori):
    K=ori[:]
    order=re.findall('\$[a-zA-Z](\d+)',ori)
    if order:
        d,ds = dimensions(ori,2)
        o=int(order[0])
        if d:
            K = ori[:ds[0].start()]+ds[-1 if len(ds) < o else o-1].group(0)+order[0]
        else:
            x_dim = list(re.finditer('[a-zA-Z]\d+\_\d+',ori))
            K= ori[:x_dim[0].start()]+x_dim[-1 if len(x_dim) < o else o-1].group(0)+order[0] if x_dim else ori
    else:
        K = ori
    return [int(c)if c.isdigit()else c for c in re.split('(\d+)',K)]

regex  =  re.compile(r'\$[a-zA-Z]\d')
regex1  =  re.compile(r'\[\d+\]')
regex2  =  re.compile(r'\[\d+\]\[\d+\]')
regex3  =  re.compile(r'x\d+\_\d+')
regexDR  =  re.compile(r'_DI_')

def findposition(m):
    for zz in range(len(m)):
        match = regex.search(m[zz])
        if match:
            return zz
def _order_key(I):
    K  =  I[:]
    match  =  regex.search(I)	# with O#
    if  match:
        y  =  regex.search(I).group(0)
        index  =  y[len(y)-1]
        m1=re.split('/',  I) # split based on /
        oposition = findposition(m1)
        if oposition==1:
            matchDR = regexDR.search(m1[oposition-1])
            if matchDR:
                m=re.split('_DI_',  m1[oposition-1]) # _DR_  	
                m2=re.split('_',  m[1])
                m3=re.split(m2[0],  m[0])
                y  =  regex.search(I).group(0)
                index  =  y[len(y)-1]
                if (int(index)==1):
                    K = m[0]
                else:
                    K = m3[0]+m[int(index)-1]
            else:
                K  =  I
        else:
            K = m1[0] + '/'
            for aa in range(1, oposition-1):
                K = K + m1[aa] + '/'
            y  =  regex.search(I).group(0)
            index  =  y[len(y)-1]
            matchDR = regexDR.search(m1[oposition-1])
            if matchDR:
                m=re.split('_DI_',  m1[oposition-1])             
                if int(index)==1:
                    K = K+ m[0]
                else:
                    K = K + m[int(index)-1]
            else:
                K  =  I
    else:       
        K  =  I
    return [int(c)if c.isdigit()else c for c in re.split('(\d+)',K)]

def order_key_method(origin_mapping):
    ori=[sorted(list(origin_mapping.keys())),sorted(list(origin_mapping.values()))]
    _ori=[[_order_key(j)for j in i] for i in ori]
    KV=[list(zip(ori[i],_ori[i])) for i in[0,1]]
    KV=[sorted(x,key=lambda x:x[1]) for x in KV]
    return [[z[0]for z in y]for y in KV]

def check_type(origin_mapping):
    output_script = ""
    choise = -1
    methods = ['Default', 'Natural', 'Order']
    functions = [default_key_method, natural_key_method, order_key_method]
    data = [ [ [] for j in range(4) ] for i in range(len(methods)) ] # index, one level run-length, readable, two level run-length
    for i in range(len(methods)):
        if choise == -1:
            input_table = functions[i](origin_mapping)   
            flag, data[i][0], data[i][1], data[i][2], data[i][3] = generate_tables(origin_mapping, input_table, methods[i])
            choise = i if flag == i else -1   
    if choise != -1:
        output_script = complete_match[choise]
    else:
        print('No method can solve this case.')
        two_level = [len("".join(str(data[i][3]).split(' ')))+len(incomplete_match[i]) for i in range(len(methods)) ]
        t_index = two_level.index(min(two_level))
        script_candidate = [ (len("".join(str(data[i][3]).split(' '))),len(incomplete_match[i]),len("".join(str(data[i][3]).split(' ')))+len(incomplete_match[i]) )for i in range(len(methods)) ]
        print('Script candidate: %s.' % str(dict(zip(methods,script_candidate))) )
        print('Using %s key table to record the original relationship.' % methods[t_index])
        index, one_level, readable, two_level = data[t_index][0], data[t_index][1], data[t_index][2], data[t_index][3] 
        #print(two_level)
        #print(incomplete_match[t_index])
        output_script = incomplete_match[t_index].format("".join(str(two_level).split(' ')))
    print('Script size: %d.' % len(output_script))
    return output_script

def test_all(case):
    print('case: {}'.format(case))
    map_in_name, script_name = '../cases/case{}.json'.format(case), 'test.py'
    map_in_file = open(map_in_name)
    origin_mapping = json.load(map_in_file)
    map_in_file.close()  
    #print('Script:\n%s\n'%check_type(origin_mapping))
    check_type(origin_mapping)
    print()
if __name__ == '__main__':
    ta = [0,1,2,3,4,5,6,7,8]
    tt=[8]
    for i in ta:
        test_all(i)
    


case: 0
Default key can sovle this case.
Script size: 105.

case: 1
Default key can't sovle this case.
Shift count in this method: 10442.
Shift table string length: 45962.
One level run-length table string length: 20453.
The most large consecutive segment length: 429.
Two level run-length table string length: 20935.
Natural key can't sovle this case.
Shift count in this method: 3274.
Shift table string length: 42299.
One level run-length table string length: 7518.
The most large consecutive segment length: 429.
Two level run-length table string length: 7362.
Order key can't sovle this case.
Shift count in this method: 3274.
Shift table string length: 42299.
One level run-length table string length: 7518.
The most large consecutive segment length: 429.
Two level run-length table string length: 7362.
No method can solve this case.
Script candidate: {'Default': (20935, 342, 21277), 'Natural': (7362, 416, 7778), 'Order': (7362, 1182, 8544)}.
Using Natural key table to record the original r

In [408]:
map_in_file = open('../cases/case{}.json'.format(1))
origin_mapping = json.load(map_in_file)
map_in_file.close()  
A=[list(origin_mapping.keys()),list(origin_mapping.keys())]
regex  =  re.compile(r'\$[a-zA-Z]\d')
regex1  =  re.compile(r'\[\d+\]')
regex2  =  re.compile(r'\[\d+\]\[\d+\]')
regex3  =  re.compile(r'x\d+\_\d+')
regexDR  =  re.compile(r'_ZN_')
regexTIU_FIGS = re.compile(r'DOH_HUCK_')

def findposition(m):
    for zz in range(len(m)):
        match = regex.search(m[zz])
        if match:
            return zz
def _order_key(I):
    K  =  I[:]
    match  =  regex.search(I)	# with O#
    if  match:
        y  =  regex.search(I).group(0)
        index  =  y[len(y)-1]
        m1=re.split('/',  I) # split based on /
        oposition = findposition(m1)
        if oposition==1:
            matchDR = regexDR.search(m1[oposition-1])
            if matchDR:
                m=re.split('_ZN_',  m1[oposition-1]) # _DR_  	
                m2=re.split('_',  m[1])
                m3=re.split(m2[0],  m[0])
                y  =  regex.search(I).group(0)
                index  =  y[len(y)-1]
                if (int(index)==1):
                    matchTIU_FIGS = regexTIU_FIGS.search(m[0])
                    if matchTIU_FIGS:
                        m2=re.split('DOH_HUCK_',  m[0])
                        K = m2[1]
                    else:
                        K = m[0]
                else:
                    matchTIU_FIGS = regexTIU_FIGS.search(m3[0])
                    if matchTIU_FIGS:
                        m2=re.split('DOH_HUCK_',  m3[0])
                        K = m2[1]+m[int(index)-1]           
                    else:
                        K = m3[0]+m[int(index)-1]
            else:
                K  =  I
        else:
            K = m1[0] + '/'
            for aa in range(1, oposition-1):
                K = K + m1[aa] + '/'
            y  =  regex.search(I).group(0)
            index  =  y[len(y)-1]
            matchDR = regexDR.search(m1[oposition-1])
            if matchDR:
                m=re.split('_ZN_',  m1[oposition-1])             
                if int(index)==1:
                    matchTIU_FIGS = regexTIU_FIGS.search(m[0])
                    if matchTIU_FIGS:
                        m2=re.split('DOH_HUCK_',  m[0])
                        K = K+ m2[1]
                    else:
                        K  =  I
                else:
                    K = K + m[int(index)-1]
            else:
                K  =  I
    else:       
        K  =  I
    return [int(c)if c.isdigit()else c for c in re.split('(\d+)',K)]
B=[ [_order_key(Y) for Y in X] for X in A]
C=[list(zip(A[X],B[X]))for X in[0,1]]
C=[sorted(x,key=lambda y:y[1])for x in C]
K=[i[0]for i in C[0]]
Y=[i[0]for i in C[1]]
count = 0
for i in range(len(K)):
    if origin_mapping[K[i]] != Y[i]:
        count += 1
print(count)

10868


In [413]:
map_in_file = open('../cases/case{}.json'.format(6))
origin_mapping = json.load(map_in_file)
map_in_file.close()  
A=[list(origin_mapping.keys()),list(origin_mapping.keys())]
regex  =  re.compile(r'\$[a-zA-Z]\d')
regex1  =  re.compile(r'\[\d+\]')
regex2  =  re.compile(r'\[\d+\]\[\d+\]')
regex3  =  re.compile(r'x\d+\_\d+')
regexDR  =  re.compile(r'_DI_')

def findposition(m):
    for zz in range(len(m)):
        match = regex.search(m[zz])
        if match:
            return zz
def _order_key(I):
    K  =  I[:]
    match  =  regex.search(I)	# with O#
    if  match:
        y  =  regex.search(I).group(0)
        index  =  y[len(y)-1]
        m1=re.split('/',  I) # split based on /
        oposition = findposition(m1)
        if oposition==1:
            matchDR = regexDR.search(m1[oposition-1])
            if matchDR:
                m=re.split('_DI_',  m1[oposition-1]) # _DR_  	
                m2=re.split('_',  m[1])
                m3=re.split(m2[0],  m[0])
                y  =  regex.search(I).group(0)
                index  =  y[len(y)-1]
                if (int(index)==1):
                    K = m[0]
                else:
                    K = m3[0]+m[int(index)-1]
            else:
                K  =  I
        else:
            K = m1[0] + '/'
            for aa in range(1, oposition-1):
                K = K + m1[aa] + '/'
            y  =  regex.search(I).group(0)
            index  =  y[len(y)-1]
            matchDR = regexDR.search(m1[oposition-1])
            if matchDR:
                m=re.split('_DI_',  m1[oposition-1])             
                if int(index)==1:
                    K = K+ m[0]
                else:
                    K = K + m[int(index)-1]
            else:
                K  =  I
    else:       
        K  =  I
    return [int(c)if c.isdigit()else c for c in re.split('(\d+)',K)]
B=[ [_order_key(Y) for Y in X] for X in A]
C=[list(zip(A[X],B[X]))for X in[0,1]]
C=[sorted(x,key=lambda y:y[1])for x in C]
K=[i[0]for i in C[0]]
Y=[i[0]for i in C[1]]
count = 0
for i in range(len(K)):
    if origin_mapping[K[i]] != Y[i]:
        count += 1
print(count)

14539
